<a href="https://colab.research.google.com/github/gc1523/iml/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3: Machine Learning Evaluation


## Version history

| Date | Author | Description |
|:----:|:------:|:------------|
2021-01-27 | Josiah Wang | First version |
2021-10-24 | Josiah Wang | Added missing random generator parameter/arguments to cross validation functions/function calls |

## Introduction

The aim of this lab exercise is to improve your understanding and to give you some practical experience in evaluating machine learning algorithms.

By the end of this lab exercise, you will have
- implemented different evaluation metrics
- performed cross-validation

## Setup

To work on this lab, we will copy some of our solutions from the previous labs.


### Dataset

We will again work with the Iris dataset.

In [1]:
import os
import numpy as np
from numpy.random import default_rng

# Download iris data if it does not exist
if not os.path.exists("iris.data"):
    !wget -O iris.data https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data


def read_dataset(filepath):
    """ Read in the dataset from the specified filepath

    Args:
        filepath (str): The filepath to the dataset file

    Returns:
        tuple: returns a tuple of (x, y, classes), each being a numpy array.
               - x is a numpy array with shape (N, K),
                   where N is the number of instances
                   K is the number of features/attributes
               - y is a numpy array with shape (N, ), and should be integers from 0 to C-1
                   where C is the number of classes
               - classes : a numpy array with shape (C, ), which contains the
                   unique class labels corresponding to the integers in y
    """

    x = []
    y_labels = []
    for line in open(filepath):
        if line.strip() != "": # handle empty rows in file
            row = line.strip().split(",")
            x.append(list(map(float, row[:-1])))
            y_labels.append(row[-1])

    [classes, y] = np.unique(y_labels, return_inverse=True)

    x = np.array(x)
    y = np.array(y)
    return (x, y, classes)


def split_dataset(x, y, test_proportion, random_generator=default_rng()):
    """ Split dataset into training and test sets, according to the given
        test set proportion.

    Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)
        y (np.ndarray): Class labels, numpy array with shape (N,)
        test_proprotion (float): the desired proportion of test examples
                                 (0.0-1.0)
        random_generator (np.random.Generator): A random generator

    Returns:
        tuple: returns a tuple of (x_train, x_test, y_train, y_test)
               - x_train (np.ndarray): Training instances shape (N_train, K)
               - x_test (np.ndarray): Test instances shape (N_test, K)
               - y_train (np.ndarray): Training labels, shape (N_train, )
               - y_test (np.ndarray): Test labels, shape (N_test, )
    """

    shuffled_indices = random_generator.permutation(len(x))
    n_test = round(len(x) * test_proportion)
    n_train = len(x) - n_test
    x_train = x[shuffled_indices[:n_train]]
    y_train = y[shuffled_indices[:n_train]]
    x_test = x[shuffled_indices[n_train:]]
    y_test = y[shuffled_indices[n_train:]]
    return (x_train, x_test, y_train, y_test)



(x, y, classes) = read_dataset("iris.data")

seed = 60012
rg = default_rng(seed)
x_train, x_test, y_train, y_test = split_dataset(x, y,
                                                 test_proportion=0.2,
                                                 random_generator=rg)
print(x_train.shape)
print(x_test.shape)


--2025-10-20 13:49:02--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘iris.data’

iris.data               [ <=>                ]   4.44K  --.-KB/s    in 0s      

2025-10-20 13:49:02 (62.0 MB/s) - ‘iris.data’ saved [4551]

(120, 4)
(30, 4)


### Classifier

We will also evaluate our random baseline classifier from Lab 1 and the K-nearest neighbours classifier from Lab 2.

In [2]:
class RandomClassifier:
    def __init__(self, random_generator=default_rng()):
        self.random_generator = random_generator
        self.unique_y = []

    def fit(self, x, y):
        """ Fit the training data to the classifier.

        Args:
            x (np.ndarray): Instances, numpy array with shape (N,K)
            y (np.ndarray): Class labels, numpy array with shape (N,)
        """
        self.unique_y = list(set(y))

    def predict(self, x):
        """ Perform prediction given some examples.

        Args:
            x (np.ndarray): Instances, numpy array with shape (N,K)

        Returns:
            y (np.ndarray): Predicted class labels, numpy array with shape (N,)
        """
        random_indices = self.random_generator.integers(0, len(self.unique_y), len(x))
        y = np.array(self.unique_y)
        return y[random_indices]


random_classifier = RandomClassifier(rg)
random_classifier.fit(x_train, y_train)
random_predictions = random_classifier.predict(x_test)
print(random_predictions)

[0 1 1 0 0 1 2 2 1 0 1 2 0 2 2 0 2 0 1 0 1 0 0 2 0 0 0 2 1 1]


In [3]:
class KNNClassifier:
    def __init__(self, k=5):
        """ K-NN Classifier.

        Args:
            k (int): Number of nearest neighbours. Defaults to 5.
        """
        self.k = k
        self.x = np.array([])
        self.y = np.array([])

    def fit(self, x, y):
        """ Fit the training data to the classifier.

        Args:
            x (np.ndarray): Instances, numpy array with shape (N,K)
            y (np.ndarray): Class labels, numpy array with shape (N,)
        """
        self.x = x
        self.y = y

    def predict(self, x):
        """ Perform prediction given some examples.

        Args:
            x (np.ndarray): Instances, numpy array with shape (N,K)

        Returns:
            y (np.ndarray): Predicted class labels, numpy array with shape (N,)
        """

        # just to make sure that we have enough training examples!
        k = min([self.k, len(self.x)])

        y = np.zeros((len(x), ), dtype=self.y.dtype)
        for (i, instance) in enumerate(x):
            distances = np.sqrt(np.sum((instance-self.x)**2, axis=1))
            sorted_indices = np.argsort(distances)
            sorted_indices = sorted_indices[:k]

            # Assign to the majority class label (the mode)
            unique_labels, freq = np.unique(self.y[sorted_indices], return_counts=True)
            y[i] = unique_labels[freq.argmax()]

        return y


knn_classifier = KNNClassifier(k=1) # we'll do one nearest neighbour
knn_classifier.fit(x_train, y_train)
knn_predictions = knn_classifier.predict(x_test)
print(knn_predictions)

[1 2 1 2 0 1 2 0 0 2 2 2 2 0 2 0 0 2 1 1 2 1 1 1 1 0 0 0 0 0]


## Evaluation metrics

Now, let us implement some functions to compute some of the metrics discussed in the lectures. For simplicity, in this tutorial we will write separate functions for each, but you can also write them as methods for a single class, for example an `Evaluator` class.

### Confusion Matrix

Firstly, given a list of predictions and the ground truth annotations, complete the function `confusion_matrix()` to compute the confusion matrix. While not a metric in itself, the confusion matrix is a useful visualisation tool for analysing the performance of your classifier across categories. You can also compute many evaluation metrics from the confusion matrix.

We will use the standard convention as shown in the lectures. The rows are the correct classes, the columns are the predicted classes.

Implementation-wise, for each of the correct classes (row), you should count how many of these instances are predicted for each class.



In [6]:
def confusion_matrix(y_gold, y_prediction, class_labels=None):
    """ Compute the confusion matrix.

    Args:
        y_gold (np.ndarray): the correct ground truth/gold standard labels
        y_prediction (np.ndarray): the predicted labels
        class_labels (np.ndarray): a list of unique class labels.
                               Defaults to the union of y_gold and y_prediction.

    Returns:
        np.array : shape (C, C), where C is the number of classes.
                   Rows are ground truth per class, columns are predictions
    """

    # if no class_labels are given, we obtain the set of unique class labels from
    # the union of the ground truth annotation and the prediction
    if not class_labels:
        class_labels = np.unique(np.concatenate((y_gold, y_prediction)))

    label_to_index = {label: i for i, label in enumerate(class_labels)}
    confusion = np.zeros((len(class_labels), len(class_labels)), dtype=int)

    for (gold, pred) in zip(y_gold, y_prediction):
      confusion[label_to_index[gold]][label_to_index[pred]] += 1

    return confusion



# Compute confusion on predictions for RandomClassifier and KNNClassifier from earlier
print("Ground truth:", y_test)

print("Random:", random_predictions)
confusion_random = confusion_matrix(y_test, random_predictions)
print(confusion_random)

print("KNN:", knn_predictions)
confusion_knn = confusion_matrix(y_test, knn_predictions)
print(confusion_knn)



Ground truth: [1 2 1 2 0 1 2 0 0 2 2 2 2 0 2 0 0 2 2 1 2 1 1 1 1 0 0 0 0 0]
Random: [0 1 1 0 0 1 2 2 1 0 1 2 0 2 2 0 2 0 1 0 1 0 0 2 0 0 0 2 1 1]
[[4 3 4]
 [5 2 1]
 [4 4 3]]
KNN: [1 2 1 2 0 1 2 0 0 2 2 2 2 0 2 0 0 2 1 1 2 1 1 1 1 0 0 0 0 0]
[[11  0  0]
 [ 0  8  0]
 [ 0  1 10]]


### Accuracy

We have already implemented the accuracy metric in Lab 1. I have copied it and renamed it to `accuracy()` just to make it more compact.

In [24]:
def accuracy(y_gold, y_prediction):
    """ Compute the accuracy given the ground truth and predictions

    Args:
        y_gold (np.ndarray): the correct ground truth/gold standard labels
        y_prediction (np.ndarray): the predicted labels

    Returns:
        float : the accuracy
    """

    assert len(y_gold) == len(y_prediction)

    try:
        return np.sum(y_gold == y_prediction) / len(y_gold)
    except ZeroDivisionError:
        return 0.

# Compute accuracy on predictions for RandomClassifier and KNNClassifier from earlier
print(accuracy(y_test, random_predictions))
print(accuracy(y_test, knn_predictions))


AssertionError: 

As an extra exercise, you can also compute the accuracy directly from a confusion matrix. This is just a different way of computing the accuracy. You can actually do this in one single line (if you disregard division by zero cases). Hint: you might have already come across the solution in my NumPy tutorial.

In [7]:
def accuracy_from_confusion(confusion):
    """ Compute the accuracy given the confusion matrix

    Args:
        confusion (np.ndarray): shape (C, C), where C is the number of classes.
                    Rows are ground truth per class, columns are predictions

    Returns:
        float : the accuracy
    """

    if np.sum(confusion) > 0:
        return np.matrix.trace(confusion) / np.sum(confusion)
    else:
        return 0.

And let's test our implementation (and check that the accuracy matches the one from above).

In [8]:
print(accuracy_from_confusion(confusion_random))
print(accuracy_from_confusion(confusion_knn))

0.3
0.9666666666666667


### Precision

Now, complete the function `precision()` to compute the **precision** given the predictions and the ground truth annotations.

To complement the equation given in the lecture, I personally find it easier to use this statement to remember the definition of precision: "*out of all instances predicted as positive, how many are correctly predicted?*" The second half of the statement is the numerator, and the first half is the denominator.

You should compute one precision score per class (so  3 scores for the Iris dataset). You might as well also compute and return the macro-averaged precision score at the same time. Also remember to account for division-by-zero cases -- for this lab we will return 0 if the denominator is 0 (Some implementations return `NaN` or `undefined`).

It is probably easier to first compute the confusion matrix and then compute the precision from the matrix (although you do not have to do this).


In [11]:
def precision(y_gold, y_prediction):
    """ Compute the precision score per class given the ground truth and predictions

    Also return the macro-averaged precision across classes.

    Args:
        y_gold (np.ndarray): the correct ground truth/gold standard labels
        y_prediction (np.ndarray): the predicted labels

    Returns:
        tuple: returns a tuple (precisions, macro_precision) where
            - precisions is a np.ndarray of shape (C,), where each element is the
              precision for class c
            - macro-precision is macro-averaged precision (a float)
    """
    class_labels = np.unique(np.concatenate((y_gold, y_prediction)))
    confusion = confusion_matrix(y_gold, y_prediction)

    p = np.zeros(len(class_labels))
    for ix, label in enumerate(class_labels):
      p[ix] = confusion[ix][ix] / np.sum(confusion[:, ix]) if np.sum(confusion[:, ix]) > 0 else 0
    # Compute the macro-averaged precision
    macro_p = np.mean(p)

    return (p, macro_p)


Test your function...

In [12]:
(p_random, macro_p_random) = precision(y_test, random_predictions)
print(p_random)
print(macro_p_random)

(p_knn, macro_p_knn) = precision(y_test, knn_predictions)
print(p_knn)
print(macro_p_knn)

[0.30769231 0.22222222 0.375     ]
0.30163817663817666
[1.         0.88888889 1.        ]
0.9629629629629629


### Recall

The next metric is **recall**. Complete the function `recall()` to compute the per-class recall (and macro-averaged recall) given the predictions and the ground truth annotations.

Again, to remember the definition of recall, I personally use this statement: "*out of all instances that are actually positive, how many are correctly retrieved?*"

This is just a small modification from computing precision. You really only need to change the denominator. So you can actually copy your solution from precision and do a few small tweaks.

In [13]:
def recall(y_gold, y_prediction):
    """ Compute the recall score per class given the ground truth and predictions

    Also return the macro-averaged recall across classes.

    Args:
        y_gold (np.ndarray): the correct ground truth/gold standard labels
        y_prediction (np.ndarray): the predicted labels

    Returns:
        tuple: returns a tuple (recalls, macro_recall) where
            - recalls is a np.ndarray of shape (C,), where each element is the
                recall for class c
            - macro-recall is macro-averaged recall (a float)
    """
    class_labels = np.unique(np.concatenate((y_gold, y_prediction)))
    confusion = confusion_matrix(y_gold, y_prediction)

    r = np.zeros(len(class_labels))
    for ix, label in enumerate(class_labels):
      r[ix] = confusion[ix][ix] / np.sum(confusion[ix]) if np.sum(confusion[ix]) > 0 else 0

    # Compute the macro-averaged recall
    macro_r = np.mean(r)

    return (r, macro_r)


Again, test your function...

In [14]:
(r_random, macro_r_random) = recall(y_test, random_predictions)
print(r_random)
print(macro_r_random)

(r_knn, macro_r_knn) = recall(y_test, knn_predictions)
print(r_knn)
print(macro_r_knn)

[0.36363636 0.25       0.27272727]
0.29545454545454547
[1.         1.         0.90909091]
0.9696969696969697


### $F_1$-score

Finally, let us compute the $F_1$-score (or $F_1$-measure). Since you have already implemented `precision()` and `recall()`, this function should be a piece of cake!

The macro-averaged $F_1$ is worth a bit more discussion though. There are actually **two** possible definitions of macro-averaged $F_1$ (although most people do not think about it).
1. Compute the mean of $F_1$ scores across the classes; or
2. Compute the *harmonic mean* between the *macro-averaged precision* and *macro-average recall*

Both definitions are used in the literature, and both give different values. See https://towardsdatascience.com/a-tale-of-two-macro-f1s-8811ddcf8f04 for a detailed discussion if you are interested.

For this lab (and our course), we will use the **first** definition.

In [17]:
def f1_score(y_gold, y_prediction):
    """ Compute the F1-score per class given the ground truth and predictions

    Also return the macro-averaged F1-score across classes.

    Args:
        y_gold (np.ndarray): the correct ground truth/gold standard labels
        y_prediction (np.ndarray): the predicted labels

    Returns:
        tuple: returns a tuple (f1s, macro_f1) where
            - f1s is a np.ndarray of shape (C,), where each element is the
              f1-score for class c
            - macro-f1 is macro-averaged f1-score (a float)
    """

    (precisions, macro_p) = precision(y_gold, y_prediction)
    (recalls, macro_r) = recall(y_gold, y_prediction)

    assert len(precisions) == len(recalls)

    f = np.zeros((len(precisions)), )
    for i in range (len(precisions)):
      f[i] = (2 * precisions[i] * recalls[i]) / (precisions[i] + recalls[i])

    macro_f = np.mean(f)

    return (f, macro_f)

And we test our function as usual.

In [18]:
(f1_random, macro_f1_random) = f1_score(y_test, random_predictions)
print(f1_random)
print(macro_f1_random)

(f1_knn, macro_f1_knn) = f1_score(y_test, knn_predictions)
print(f1_knn)
print(macro_f1_knn)

[0.33333333 0.23529412 0.31578947]
0.29480564155486755
[1.         0.94117647 0.95238095]
0.9645191409897292


And that is all the metrics that you will implement! You can use these for your first coursework to evaluate your classifiers.

## Cross-validation

So far, we have evaluated our classifiers by splitting our dataset into a *training set* and a *test set*.

One of the issues with using a *single* test set is that it is sometimes difficult to definitively conclude that one model  is better than another. Your model might just so happen to perform better than another on one particular test set, and
the opposite might happen if evaluated on another. Therefore, **k-fold cross-validation** is commonly
performed to ensure that the better performance is not just by chance, but is consistent across different
data splits.



### Divide dataset into $k$ splits

Below is a utility function to randomly divide a dataset into $k$ approximately equal-sized splits so that we can use these splits for cross-validation.

The function will return the *indices* rather than the dataset itself. Otherwise you will have to split `x` and `y` simultaneously and make sure they correspond to each other correctly. We can later access the appropriate `x` and `y` using these indices.

In [19]:
def k_fold_split(n_splits, n_instances, random_generator=default_rng()):
    """ Split n_instances into n mutually exclusive splits at random.

    Args:
        n_splits (int): Number of splits
        n_instances (int): Number of instances to split
        random_generator (np.random.Generator): A random generator

    Returns:
        list: a list (length n_splits). Each element in the list should contain a
            numpy array giving the indices of the instances in that split.
    """

    # generate a random permutation of indices from 0 to n_instances
    shuffled_indices = random_generator.permutation(n_instances)

    # split shuffled indices into almost equal sized splits
    split_indices = np.array_split(shuffled_indices, n_splits)

    return split_indices

# For quick testing
k_fold_split(3, 20)

[array([15,  3, 19,  8, 14, 17,  7]),
 array([13,  1,  6, 11,  4, 10, 18]),
 array([ 0,  2,  5, 12,  9, 16])]

### $k$-fold cross validation

Now, complete the `train_test_k_fold` function below. This function will generate indices for train and test splits for each fold. For example, if there are 20 instances and 3 folds, you should first divide the 20 instances into 3 splits:

```python
[array([19,  6, 14, 13, 17, 12,  5]),
 array([ 1,  2,  0, 10, 18, 15, 16]),
 array([ 4,  3,  8,  9,  7, 11])]
 ```

You will then produce 3 train/test splits for each fold, using one of the splits for testing each time, and the remaining two splits for training. For example:

```python
[[array([ 1,  2,  0, 10, 18, 15, 16, 4,  3,  8,  9,  7, 11]), array([19,  6, 14, 13, 17, 12,  5])]
 [array([19,  6, 14, 13, 17, 12,  5, 4,  3,  8,  9,  7, 11]), array([1,  2,  0, 10, 18, 15, 16])]
 [array([19,  6, 14, 13, 17, 12,  5, 1,  2,  0, 10, 18, 15]), array([16, 4,  3,  8,  9,  7, 11])]
]
```


In [20]:
def train_test_k_fold(n_folds, n_instances, random_generator=default_rng()):
    """ Generate train and test indices at each fold.

    Args:
        n_folds (int): Number of folds
        n_instances (int): Total number of instances
        random_generator (np.random.Generator): A random generator

    Returns:
        list: a list of length n_folds. Each element in the list is a list (or tuple)
            with two elements: a numpy array containing the train indices, and another
            numpy array containing the test indices.
    """

    # split the dataset into k splits
    split_indices = k_fold_split(n_folds, n_instances, random_generator)

    folds = []
    for k in range(n_folds):
        # TODO: Complete this
        # take the splits from split_indices and keep the k-th split as testing
        # and concatenate the remaining splits for training

        test_indices = split_indices[1]
        train_indices = split_indices[0]

        folds.append([train_indices, test_indices])

    return folds


# to test your function (30 instances, 4 fold)
for (train_indices, test_indices) in train_test_k_fold(4, 30, rg):
    print("train: ", train_indices)
    print("test: ", test_indices)
    print()

train:  [24  0 11 20 13 29 21 15]
test:  [16 28  3  9  6 18 25  1]

train:  [24  0 11 20 13 29 21 15]
test:  [16 28  3  9  6 18 25  1]

train:  [24  0 11 20 13 29 21 15]
test:  [16 28  3  9  6 18 25  1]

train:  [24  0 11 20 13 29 21 15]
test:  [16 28  3  9  6 18 25  1]



You will now train and evaluate the K-NN classifier with 10-fold cross validation, using the original (unsplit) dataset (i.e. `x`, `y`). Compute the **accuracy** for each fold, and return the average accuracy and standard deviation across all folds. You will likely see that some of the accuracies are higher than others. You can set the number of nearest neighbours for the K-NN classifier to be any number.

In [29]:
n_folds = 10
accuracies = np.zeros((n_folds, ))
for i, (train_indices, test_indices) in enumerate(train_test_k_fold(n_folds, len(x), rg)):
    # Get the dataset from the correct splits
    # TODO: Complete test
    x_train = x[train_indices, :]
    y_train = y[train_indices]
    x_test = x[test_indices, :]
    y_test = y[test_indices]

    knn_classifier = KNNClassifier(k=1)
    knn_classifier.fit(x_train, y_train)
    predictions = knn_classifier.predict(x_test)
    acc = accuracy(y_test, predictions)
    accuracies[i] = acc

print(accuracies)
print(accuracies.mean())
print(accuracies.std())

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1.0
0.0


### $k$-fold cross validation with hyperparameter tuning

If your classifier has some hyperparameters (e.g. the number of neighbours $K$ for K-NN), you can divide your dataset into three disjoint sets: *training*, *validation* and *test*, and use the validation set to select the optimal hyperparamter value ($K$ in the case of K-NN). We will not implement this in this tutorial since this is a straightforward modification of the `split_dataset()` function from Lab 1 (try this yourself if you are so inclined!)

We will instead focus on selecting the optimal hyperparameter values using $k$-fold cross-validation. This will ensure that you do not overestimate the accuracy of your algorithm. More specifically, you will implement cross-validation that includes a validation dataset as presented in the lecture. You will implement the version labelled as **Option 1**, which is much simpler than Option 2. To do this, for each fold, keep one split for testing, one split for validation, and the remaining splits for training.

For this, you will need to first modify the `train_test_k_fold()` function from earlier. Call the function `train_val_test_k_fold()`, and it should now return the train, validation and test splits per fold instead of just train and test. For each fold, reserve one split as the test set, pick any one of the other splits as the validation set, and put the remaining splits together as the training set.

In [31]:
def train_val_test_k_fold(n_folds, n_instances, random_generator=default_rng()):
    """ Generate train and test indices at each fold.

    Args:
        n_folds (int): Number of folds
        n_instances (int): Total number of instances
        random_generator (np.random.Generator): A random generator

    Returns:
        list: a list of length n_folds. Each element in the list is a list (or tuple)
            with three elements:
            - a numpy array containing the train indices
            - a numpy array containing the val indices
            - a numpy array containing the test indices
    """

    # split the dataset into k splits
    split_indices = k_fold_split(n_folds, n_instances, random_generator)

    folds = []
    for k in range(n_folds):
        # TODO: Complete this
        # take the splits from split_indices and keep the k-th split as testing
        # and another split as validation
        # and concatenate the remaining k-2 splits for training
        test_indices = split_indices[k]
        val_indices = split_indices[(k+1) % n_folds]
        train_indices = np.zeros((0, ), dtype=int)
        for i in range(n_folds):
            if i not in [k, (k+1) % n_folds]:
                train_indices = np.hstack([train_indices, split_indices[i]])

        folds.append([train_indices, val_indices, test_indices])

    return folds


# to test your function (30 instances, 4 fold)
for (train_indices, val_indices, test_indices) in train_val_test_k_fold(4, 30, rg):
    print("train: ", train_indices)
    print("validation: ", val_indices)
    print("test: ", test_indices)
    print()

train:  [21 29  1 24  8 26  7  0 18  3 27  5 28 16]
validation:  [17 13  9 20 14  6 19 25]
test:  [11 12 10  4 22  2 23 15]

train:  [11 12 10  4 22  2 23 15  0 18  3 27  5 28 16]
validation:  [21 29  1 24  8 26  7]
test:  [17 13  9 20 14  6 19 25]

train:  [11 12 10  4 22  2 23 15 17 13  9 20 14  6 19 25]
validation:  [ 0 18  3 27  5 28 16]
test:  [21 29  1 24  8 26  7]

train:  [17 13  9 20 14  6 19 25 21 29  1 24  8 26  7]
validation:  [11 12 10  4 22  2 23 15]
test:  [ 0 18  3 27  5 28 16]



We will now implement cross-validation with hyperparameter tuning. For each fold, you will perform what is known as a **grid search** to exhaustively search for the hyperparameter value that optimises the performance of the model on the validation set. You should then evaluate this model on the held-out test set, and average the score across folds.

For this exercise, you will try to optimise the **number of nearest neighbours** in a K-NN classifier, using the **accuracy** metric. For simplicity, just perform a grid search over 1 to 10 nearest neighbours.

In [34]:
n_folds = 10
accuracies = np.zeros((n_folds, ))
for i, (train_indices, val_indices, test_indices) in enumerate(train_val_test_k_fold(n_folds, len(x), rg)):
    # set up the dataset for this fold
    x_train = x[train_indices, :]
    y_train = y[train_indices]
    x_val = x[val_indices, :]
    y_val = y[val_indices]
    x_test = x[test_indices, :]
    y_test = y[test_indices]

    hyp_accuracies = []
    for nn in range(1, 11):
        knn_classifier = KNNClassifier(k=nn)
        knn_classifier.fit(x_train, y_train)
        predictions = knn_classifier.predict(x_val)
        acc = accuracy(y_val, predictions)
        hyp_accuracies.append((acc, nn, knn_classifier))
    (best_acc, best_nn, best_classifier) = max(hyp_accuracies, key=lambda x:x[0])
    predictions = best_classifier.predict(x_test)
    acc = accuracy(y_test, predictions)
    accuracies[i] = acc

print(accuracies)
print(accuracies.mean())
print(accuracies.std())

[1.         1.         0.93333333 0.93333333 1.         1.
 0.93333333 0.93333333 0.93333333 1.        ]
0.9666666666666666
0.033333333333333326


And that is it for cross-validation. "Option 2" from the lecture (nested cross-validation) is less prone to overfitting, but is much slower and much more complicated to implement, so we will not attempt it for this lab tutorial. There are multiple ways to design this; I will provide my implementation for Option 2 in the solutions, so that you can have a better understanding of it.

## Summary

Phew! This was an intense exercise. Congratulations! You have managed to implement most of the major metrics for classification. You have also implemented a simple form of cross-validation (with only training and testing splits) and performed hyperparameter tuning using cross-validation.

You have hopefully gained a more in-depth understanding of cross-validation and the metrics. In practice, these can be quite tricky to implement from scratch (especially cross-validation), so you can just use the utilities provided by the scikit-learn library to easily perform cross-validation and grid search:
https://scikit-learn.org/stable/modules/cross_validation.html
